# Sunblock: Data Processing

1. Load Data from SQLite Database into a Dataframe
2. Convert Date string to Datetime object
3. Use dataframe to perform any further math 

In [1]:
#imports
import sqlite3
import datetime
import pandas as pd 
import plotly.express as px
import plotly.offline as pyo
from statistics import mean
import numpy as np
pyo.init_notebook_mode(connected=True)

In [2]:
# Constants 
DB_NAME = "./sunblockone_06-05-2025-2249.db"
DB_TABLE_NAME = "solardata"
DB_CONNECTION = None
DB_CURSOR = None 

DATESTRING_FORMAT = '%Y-%m-%d %H:%M:%S'
DATAFRAME = None


## Columns 
Timestamp = "Timestamp"
PVVoltage = "PVVoltage"
PVCurrent = "PVCurrent"
PVPower = "PVPower"
BattVoltage = "BattVoltage"
BattChargeCurrent = "BattChargeCurrent"
BattChargePower = "BattChargePower"
LoadPower = "LoadPower"
BattPercentage = "BattPercentage"
BattOverallCurrent = "BattOverallCurrent"
CPUPowerDraw = "CPUPowerDraw"
PowerProfile = "PowerProfile"

In [3]:
# Methods 
def init_db_connection():
    global DB_CONNECTION, DB_CURSOR
    DB_CONNECTION = sqlite3.connect(DB_NAME)
    DB_CURSOR = DB_CONNECTION.cursor()

def check_db_connection():
    if DB_CONNECTION != None and DB_CURSOR != None:
        return True
    return False

def insert(data):
    global DB_CURSOR, DB_CONNECTION
    
    if check_db_connection():
        db_query = "INSERT INTO " + DB_TABLE_NAME + " VALUES " + data
        DB_CURSOR.execute(db_query)
        DB_CONNECTION.commit()
    else:
        print("Error: DB Connection not Initialized")

def parsefloat(str_val):
    if isinstance(str_val, float): 
        return str_val
    if str_val:
        if str_val[0] == ".":
            return float(str_val[1:])
        if str_val[-1] == ".":
            return float(str_val[0:len(str_val)-1])
        return float(str_val)

def load_solardata(start_date=None, end_date=None):

    # Load data from DB and parse it into a dictionary containing call columns. 
    solar_data = {Timestamp: [], PVVoltage: [], PVCurrent: [], PVPower: [],
               BattVoltage: [], BattChargeCurrent: [], BattChargePower: [],
               LoadPower: [], BattPercentage: [], BattOverallCurrent: [], CPUPowerDraw: [], PowerProfile: []}
    
    max_thresholds = {
        PVVoltage: 25,
        PVCurrent: 5, 
        PVPower: 100, 
        BattVoltage: 15,
        
    }
                  
    if check_db_connection():
        query = DB_CURSOR.execute("SELECT * FROM " + DB_TABLE_NAME)
        names = [description[0] for description in DB_CURSOR.description]
            
        for row in query:
            
            #timestamp
            solar_data[Timestamp].append(datetime.datetime.strptime(row[0], DATESTRING_FORMAT))

            #PV Voltage Lowpass filtering
            if parsefloat(row[1]) > max_thresholds[PVVoltage]:
                solar_data[PVVoltage].append(solar_data[PVVoltage][-1])
            else:
                solar_data[PVVoltage].append(parsefloat(row[1]))

            #PVCurrent Lowpass filtering
            if parsefloat(row[2]) > max_thresholds[PVCurrent]:
                solar_data[PVCurrent].append(solar_data[PVCurrent][-1])
            else:
                solar_data[PVCurrent].append(parsefloat(row[2]))

            #PVPower Lowpass Filtering 
            if parsefloat(row[3]) > max_thresholds[PVPower]:
                solar_data[PVPower].append(solar_data[PVPower][-1])
            else:
                solar_data[PVPower].append(parsefloat(row[3]))

            #BattVoltage Lowpass Filtering 
            if parsefloat(row[4]) > max_thresholds[BattVoltage]:
                solar_data[BattVoltage].append(solar_data[BattVoltage][-1])
            else:
                solar_data[BattVoltage].append(parsefloat(row[4]))

            solar_data[BattChargeCurrent].append(row[5])
            solar_data[BattChargePower].append(row[6])
            solar_data[LoadPower].append(row[7])
            solar_data[BattPercentage].append(row[8])
            solar_data[BattOverallCurrent].append(row[9])
            solar_data[CPUPowerDraw].append(row[10])
            solar_data[PowerProfile].append(row[11])
    else:
        print("Error: DB Connection not Initialized")

    return pd.DataFrame(solar_data)


def cleanup():
    if check_db_connection():
        DB_CONNECTION.close()


In [4]:
# Window functions 
def window(start_date, end_date=None):
    # input: start and end dates to be Timestamp objects. 
    # output: returns tuple with indices to slice Dataframe at
    
    slice_start_index = -1
    slice_end_index = len(DATAFRAME[Timestamp]) 
    
    for i in range(len(DATAFRAME[Timestamp])):
        
        if slice_start_index < 0 and DATAFRAME[Timestamp][i].date() >= start_date:
            slice_start_index = i
            if not end_date:
                return (slice_start_index, len(DATAFRAME[Timestamp])) 
        
        if slice_start_index > -1 and DATAFRAME[Timestamp][i].date() > end_date:
            slice_end_index = i - 1 
            return (slice_start_index, slice_end_index)
    return (slice_start_index, slice_end_index)


def resample(df, step):
    # input: takes a dataframe and sampling frequency
    # returns a new dataframe 
    return df.iloc[0: len(df):step]

## Loading data from Database

In [5]:
init_db_connection()
DATAFRAME = load_solardata()
cleanup()

In [6]:
SAMPLING_FREQUENCY = 60  # in seconds

In [58]:
start_date = datetime.date(2025, 5, 6)
end_date = datetime.date(2025, 5, 6)
df_slice = window(start_date, end_date)
resampled_df = resample(DATAFRAME.loc[df_slice[0]:df_slice[1]], SAMPLING_FREQUENCY)

## Plotting with Plotly

In [ ]:
# Using plotly.express
# Plotly docs 
# https://plotly.com/python/line-charts/

In [61]:
fig = px.line(resampled_df, x=Timestamp, y=[BattPercentage])
fig.update_traces(line_color='#006600')
fig.show()


In [17]:
fig = px.line(resampled_df, x=Timestamp, y=[PVPower])
fig.update_traces(line_color='#ff0000')
fig.show()
print("Maximum Power Generated for period was", max(resampled_df[PVPower]), "watts") 
print("Total Energy Generated for period was", sum(resampled_df[PVPower]) * SAMPLING_FREQUENCY, "Joules") 
print("Average Power Generated for period was", mean(resampled_df[PVPower]), "watts")

Maximum Power Generated for period was 72.14 watts
Total Energy Generated for period was 5254396.8 Joules
Average Power Generated for period was 12.067421799641725 watts


In [12]:
fig = px.line(resampled_df, x=Timestamp, y=[LoadPower])
fig.update_traces(line_color='#008800')
fig.show()
print("Average Power draw for period was", mean(resampled_df[LoadPower]), "watts" ) 

Average Power draw for period was 11.03492627807634 watts


## CSV Operations

In [ ]:
# CSV Functions 
def read_file(filename="file.csv"):
    readings = []
    reading = {"Timestamp": "", "PVVoltage": 0, "PVCurrent": 0, "PVPower": 0,
               "BattVoltage": 0, "BattChargeCurrent": 0, "BattChargePower": 0,
               "LoadPower": 0, "BattPercentage": 0, "BattOverallCurrent": 0, "CPUPowerDraw": 0}

    _file = open(filename)

    for i in _file:
        line = i.strip().split(",")
        if ("timestamp" not in line):
            for n in range(len(line)):
                if (n == 0):
                    reading["Timestamp"] = line[n]
                else:
                    reading[list(reading.keys())[n]] = float(line[n])
            write_to_readings(reading, readings)
    _file.close()

    return readings

In [ ]:
CSV_DATA = read_file("sample.csv")

for i in CSV_DATA:
    a = str(tuple(i.values()))
    insert(a)